In [ ]:
## 필요한 패키지를 불러옵니다.
from langchain.chat_models import ChatOpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
from dotenv import load_dotenv

In [ ]:
## .env에 있는 보안(api_key) 키를 가져옵니다.
load_dotenv()

In [ ]:
## PDF 파일을 불러와 페이지 단위로 쪼갭니다.
loader = PyPDFDirectoryLoader('./pdf/')
raw_text = loader.load_and_split()

In [ ]:
## 페이지가 잘 구분되었나 일부를 확인해봅니다.
raw_text[:100]

In [ ]:
## 페이지 단위로 쪼개진 문서를 더욱 작게 쪼개봅니다. 맥락 파악을 위해 겹치는 부분이 생기도록 합니다.
def split_text(raw_text):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500,
        chunk_overlap=200,
        encoding_name='cl100k_base'  
    )

    text = text_splitter.split_documents(raw_text)
    return text


texts = split_text(raw_text)

In [ ]:
## 문서가 몇 개인지 살펴봅니다.
len(texts)

In [ ]:
## 5번째 문서만 봅니다. (0부터 시작하기 때문)
texts[4]

In [ ]:
## 여기서 과금이 시작됩니다. 여러분이 쪼개신 문서가 숫자로 바뀌어 저장소에 저장되는 작업이 OpenAI 회사를 통해 이루어집니다.
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(
    texts[:10], # 추후 전문을 추가할 때는 이 부분을 수정해주세요.
    embedding=embedding,
    distance_strategy=DistanceStrategy.COSINE # 코사인 유사도를 사용하여 문서를 비교합니다.
)

In [ ]:
## 저장소를 컴퓨터에 폴더로 저장합니다.
vectorstore.save_local('./faiss_data')